# **Setup**

In [61]:
import pandas as pd
# import re
# pip install nltk
# import nltk
# nltk.download()
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from string import punctuation
punctuation = list(punctuation)
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

# **Import and Clean Data**

## Netflix, DisneyPlus, and Amazon Prime

In [62]:
# Import the Netflix, DisneyPlus, and Amazon Prime datasets.
df_netflix = pd.read_csv("data/netflix_titles.csv");
df_disneyplus = pd.read_csv("data/disney_plus_titles.csv");
df_amazonprime = pd.read_csv("data/amazon_prime_titles.csv");

# All of these datasets have identical structure. Thus, it is easy to combine them together with a new column containing what the streaming service the title belongs to.
df_netflix["Streaming Service"] = "Netflix";
df_disneyplus["Streaming Service"] = "Disney Plus";
df_amazonprime["Streaming Service"] = "Amazon Prime";

# Combine Netflix, DisneyPlus, and Amazon Prime dataframes together.
df_temp = pd.concat([df_netflix, df_disneyplus, df_amazonprime]);
df_temp = df_temp.reset_index();

# Filter dataframe so that only necessary columns are considered for cleaning.
df_temp = df_temp[["type","title","release_year","rating","listed_in","description","Streaming Service"]];

Data Cleaning of Netflix, DisneyPlus, and Amazon Prime data

In [63]:
# Data cleaning: rating column
for i in df_temp.index:
    if (df_temp.loc[i, "rating"] == '74 min') | (df_temp.loc[i, "rating"] == '84 min') | (df_temp.loc[i, "rating"] == '66 min'):
        df_temp.loc[i, "rating"] = 'UNRATED';
df_temp["rating"] = df_temp["rating"].fillna("UNRATED");

# Data cleaning: genres column
df_temp = df_temp.rename(columns={"listed_in":"genres"});

In [64]:
df_temp[0:1]

,type,title,release_year,rating,genres,description,Streaming Service
0,Movie,Dick Johnson Is Dead,2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",Netflix


## Paramount

In [65]:
df_paramount = pd.read_csv("data/paramount_titles.csv");
df_paramount = df_paramount[["type","title","release_year","age_certification","genres","description"]];
df_paramount["Streaming Service"] = "Paramount";    

Data Cleaning of Paramount data

In [66]:
# Data cleaning: type column:
df_paramount["type"] = df_paramount["type"].replace(to_replace ="MOVIE", value ="Movie");
df_paramount["type"] = df_paramount["type"].replace(to_replace ="SHOW", value ="TV Show");

# Data cleaning: rating column:
df_paramount = df_paramount.rename(columns={"age_certification":"rating"});
df_paramount["rating"] = df_paramount["rating"].fillna("UNRATED");

In [67]:
df_paramount[0:1]

,type,title,release_year,rating,genres,description,Streaming Service
0,Movie,The General,1926,UNRATED,"['action', 'drama', 'war', 'western', 'comedy'...","During America’s Civil War, Union spies steal ...",Paramount


## Apple TV+

In [68]:
df_appletv = pd.read_csv("data/All_Streaming_Shows_apple.csv");
df_appletv = df_appletv[df_appletv["Streaming Platform"] == "Apple TV+"];
df_appletv = df_appletv[["Series Title","Year Released","Content Rating","Genre","Description","Streaming Platform"]];

Data Cleaning of Apple TV data

In [69]:
# Data cleaning: type
# The No of Seasons column (which is now a dropped column) indicate that all titles in the dataset are shows (i.e., unique values consist of just '1 season' and '1season'), and so the type is "TV Show" for all rows
df_appletv["type"] = "TV Show";

# Data cleaning: title
df_appletv = df_appletv.rename(columns={"Series Title":"title"});

# Data cleaning: year_released
df_appletv = df_appletv.rename(columns={"Year Released":"release_year"});

# Data cleaning: rating
df_appletv = df_appletv.rename(columns={"Content Rating":"rating"});

# Data cleaning: genres
df_appletv = df_appletv.rename(columns={"Genre":"genres"});
df_appletv["genres"] = df_appletv["genres"].replace(to_replace ="Comedy,2020", value ="Comedy");
df_appletv["genres"] = df_appletv["genres"].replace(to_replace ="2019,Apple TV+", value ="Talk Show");
df_appletv["genres"] = df_appletv["genres"].replace(to_replace ="Children,2020", value ="Children");
df_appletv["genres"] = df_appletv["genres"].replace(to_replace ="2020,Apple TV+", value ="Talk Show");

# Data cleaning: description
df_appletv = df_appletv.rename(columns={"Description":"description"});

# Data cleaning: Streaming Service
df_appletv = df_appletv.rename(columns={"Streaming Platform":"Streaming Service"});

In [70]:
df_appletv = df_appletv[["type","title","release_year","rating","genres","description","Streaming Service"]];
df_appletv[0:1]

,type,title,release_year,rating,genres,description,Streaming Service
130,TV Show,Ted Lasso,2020,18+,"Comedy,Drama","Ted Lasso, an American football coach, moves t...",Apple TV+


# **Data Transformation**

## Combine all cleaned datasets together

In [71]:
df_combined = pd.concat([df_temp, df_paramount, df_appletv]);
df_combined = df_combined.reset_index();
df_combined = df_combined[df_combined.columns.tolist()[1:]];
df_combined

,type,title,release_year,rating,genres,description,Streaming Service
0,Movie,Dick Johnson Is Dead,2020,PG-13,Documentaries,"As her father nears the end of his life, filmm...",Netflix
1,TV Show,Blood & Water,2021,TV-MA,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t...",Netflix
2,TV Show,Ganglands,2021,TV-MA,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,Netflix
3,TV Show,Jailbirds New Orleans,2021,TV-MA,"Docuseries, Reality TV","Feuds, flirtations and toilet talk go down amo...",Netflix
4,TV Show,Kota Factory,2021,TV-MA,"International TV Shows, Romantic TV Shows, TV ...",In a city of coaching centers known to train I...,Netflix
...,...,...,...,...,...,...,...
22771,TV Show,Dear…,2020,18+,"Documentary,Biography",An inventive and cinematic approach to biograp...,Apple TV+
22772,TV Show,Fraggle Rock: Rock On!,2020,all,"Family,Children","The Fraggles might be apart in separate caves,...",Apple TV+
22773,TV Show,Oprah's Book Club,2019,7+,Talk Show,Get a front row seat to unguarded conversation...,Apple TV+
22774,TV Show,Helpsters Help You,2020,all,Children,Cody is a helpful monster who knows that every...,Apple TV+


# ***NLP Preparation***

In [72]:
df_prep = df_combined.copy();

# Tokenization: Splits text into pieces (tokens), remove punctuation
# Stopword Removal: Removes commonly used words (such as 'the') which are not relevant to analysis
# Stemming and Lemmatization: Reduces words to base form to be analyzed as a single item
# P.O.S Tagging: Tags words to be part of speech (such as verb, noun) based on definition and context
# Information Retrieval: Extracts relevant information from source

In [73]:
df_prep["Textual Info"] = "";
for i in df_prep.index:
    textualInfo = (str(df_prep.loc[i, "rating"]) + " " + str(df_prep.loc[i, "genres"]) + " " + str(df_prep.loc[i, "description"])).lower();
    textualInfo_tokens = word_tokenize(textualInfo);
    textualInfo_cleanedTokens = [];
    for token in textualInfo_tokens:
        token = lemmatizer.lemmatize(token, pos="a");
        # if (token not in stop_words) and (token not in punctuation):
        if (token not in stop_words) and (token not in punctuation) and (token not in textualInfo_cleanedTokens):
            textualInfo_cleanedTokens.append(token);
    textualInfo_final = "";
    for token in textualInfo_cleanedTokens:
        textualInfo_final += token + " ";
    textualInfo_final = textualInfo_final[:-1];
    df_prep.loc[i, "Textual Info"] = textualInfo_final;

df_prep = df_prep.sort_values(by=["title","release_year","rating","genres","Streaming Service"], ascending=True);
df_prep = df_prep.reset_index();
df_prep = df_prep[df_prep.columns.tolist()[1:]];

In [74]:
df_prep.head()

,type,title,release_year,rating,genres,description,Streaming Service,Textual Info
0,Movie,"""Mixed Up""",2020,16+,"Drama, Romance","""Mixed Up"" examines casual factors that make u...",Amazon Prime,16+ drama romance `` mixed '' examines casual ...
1,TV Show,"""The Paramedic Angel""",2021,ALL,Drama,The tragedy of a loving family man and paramed...,Amazon Prime,drama tragedy loving family man paramedic hero...
2,Movie,#Alive,2020,TV-MA,"Horror Movies, International Movies, Thrillers","As a grisly virus rampages a city, a lone man ...",Netflix,tv-ma horror movies international thrillers gr...
3,Movie,#AnneFrank - Parallel Stories,2019,TV-14,"Documentaries, International Movies","Through her diary, Anne Frank's story is retol...",Netflix,tv-14 documentaries international movies diary...
4,Movie,#FriendButMarried,2018,TV-G,"Dramas, International Movies, Romantic Movies","Pining for his high school crush for years, a ...",Netflix,tv-g dramas international movies romantic pini...


# **Data Loading**

In [75]:
df_prep.to_excel("streaming_service_titles_TFIDF.xlsx", index=False);